# Week 1 Practical Aspect of Deep Learning

## Quiz

If your Neural Network model seems to have high bias, what of the following would be promising things to try? (Check all that apply.)

Correct:
1. Increasing numbers of units in each hidden layer
2. Make the neuron network deeper

Wrong:
Get more training data

## Initialization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
from public_tests import *
from init_utils import sigmoid, relu, compute_loss, forward_propagation, backward_propagation
from init_utils import update_parameters, predict, load_dataset, plot_decision_boundary, predict_dec

%matplotlib inline
plt.rcParams['figure.figsize'] = (7.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

# load image dataset: blue/red dots in circles
# train_X, train_Y, test_X, test_Y = load_dataset()